In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import scipy.stats as stats
from sklearn.feature_selection import RFE

from xgboost import XGBClassifier

from mpl_toolkits.mplot3d import Axes3D
from IPython.display import Image
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import plot_confusion_matrix
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
import scipy

import types
import re
import string
import nltk
import xgboost

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV

import sys

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, plot_confusion_matrix

from keras.preprocessing import text, sequence
from textblob import TextBlob, Word
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../../../data/thanos.csv")
df.head(3)
#X = df.drop(col, axis=1)
#y = df[["voted_up"]]
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=57, test_size = 0.5)


,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198271088129,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",good models\n,True,True
1,76561198138206834,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",I completely suck at making anything on here a...,True,True
2,76561198128760839,70,Half-Life,"['FPS', 'Sci-fi', 'Action', 'Singleplayer', ""1...",There is not much to say about this old game t...,True,True


In [3]:
#df = pd.concat([X_train, y_train], axis=1)
#df_test = pd.concat([X_test, y_test], axis=1)

In [4]:
#df_test.info()
def ScoreModel(model, X, y):
    preds = model.predict(X)
    acc = accuracy_score(y, preds)
    f1 = f1_score(y, preds)
    recall = recall_score(y, preds)
    precision = precision_score(y, preds)
    rockout = roc_auc_score(y, preds)
    
    print("Accuracy:  ", acc)
    print("F1 Score:  ", f1)
    print("Recall:    ", recall)
    print("Precision: ", precision)
    print("ROC_AUC:   ", rockout)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23183 entries, 0 to 23182
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    23183 non-null  int64 
 1   appid      23183 non-null  int64 
 2   app_title  23183 non-null  object
 3   app_tags   23183 non-null  object
 4   review     23183 non-null  object
 5   fps        23183 non-null  bool  
 6   voted_up   23183 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 951.0+ KB


In [6]:
#cols = ["review_id", "year", "title"]

#df = df.drop(cols, axis = 1)
#df_test= df_test.drop(cols, axis = 1)


In [7]:
df["voted_up"].value_counts(normalize=True)

True     0.505888
False    0.494112
Name: voted_up, dtype: float64

In [8]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/elliott/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/elliott/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
def CleanText(reviews):
    stopwords = nltk.corpus.stopwords.words('english')
    punct = string.punctuation
    lemma = nltk.WordNetLemmatizer()
    
    reviews = "".join([word for word in reviews if word not in string.punctuation])
    tokens = re.split('\W+', reviews)
    reviews = [lemma.lemmatize(word) for word in tokens if word not in stopwords]
    
# use function like this in your notebook
# df['cleaned_text'] = df['text'].apply(lambda x: CleanText(x.lower()))
    
    return reviews


In [10]:
def clean_reviews(reviews):
    reviews = "".join([word for word in reviews if word not in string.punctuation])
    tokens = re.split('\W+', reviews)
    reviews = [lemma.lemmatize(word) for word in tokens if word not in stopwords]
    return reviews

In [11]:
df['clean_review'] = df['review'].apply(lambda x: CleanText(x.lower()))

df.head()

,steamid,appid,app_title,app_tags,review,fps,voted_up,clean_review
0,76561198271088129,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",good models\n,True,True,"[good, model, ]"
1,76561198138206834,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",I completely suck at making anything on here a...,True,True,"[completely, suck, making, anything, death, ru..."
2,76561198128760839,70,Half-Life,"['FPS', 'Sci-fi', 'Action', 'Singleplayer', ""1...",There is not much to say about this old game t...,True,True,"[much, say, old, game, hasnt, said, really, ma..."
3,76561198079636858,6060,"Star Wars: Battlefront 2 (Classic, 2005)","['Action', 'Multiplayer', 'Shooter', 'Third-Pe...",360 noscoped almost everything 10/10 even the ...,True,True,"[360, noscoped, almost, everything, 1010, even..."
4,76561198238100200,220,Half-Life 2,"['FPS', 'Action', 'Sci-fi', 'Classic', 'Single...",No need for a review. It's practically history.,True,True,"[need, review, practically, history]"


In [12]:
#df['clean_review'] = df['review'].apply(lambda x: clean_reviews(x.lower()))

#df.head(10)

In [13]:
#col = ["voted_up"]
#X = df.drop(col, axis=1)
#y = df[["voted_up"]]

#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=57, test_size = 0.54)

In [14]:
#new_df = pd.concat([X_train, y_train], axis=1)
#new_df.head(10)

In [15]:
# = new_df.loc[new_df['voted_up'] == 0]
#df_up = new_df.loc[new_df['voted_up'] == 1]

In [16]:
#wordcloud = WordCloud(max_words=100, width=400, height=200).generate(str(df_up["clean_review"]))
#plt.imshow(wordcloud, interpolation='bilinear')
#plt.axis('off')
#plt.figure(figsize=(20,10))
#plt.show()

In [17]:
#wordcloud = WordCloud(max_words=100, width=400, height=200).generate(str(df_down["clean_review"]))
#plt.imshow(wordcloud, interpolation='bilinear')
#plt.axis('off')
#plt.figure(figsize=(20,10))
#plt.show()

In [18]:
from sklearn.naive_bayes import MultinomialNB

In [19]:
vector = TfidfVectorizer(analyzer=CleanText, ngram_range=(2, 2))
X = vector.fit_transform(df["review"])

X_df = pd.DataFrame(X.toarray())
X_df.columns = vector.get_feature_names()
X_df.head(5)

,,0,00,000,0000000000,00000000000,000000000000,0000000000000000,0000000000000000000,00000000000000000000,...,𝚝𝚑𝚎,𝚝𝚑𝚒𝚜,𝚝𝚑𝚛𝚎𝚎𝚙𝚊𝚛𝚝,𝚝𝚘,𝚠𝚊𝚗𝚗𝚊,𝚠𝚊𝚗𝚝,𝚠𝚑𝚊𝚝,𝚠𝚘𝚛𝚕𝚍,𝚢𝚘𝚞𝚛,𝟣
0,0.43771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#X = vector.fit_transform(df['review'])
y = df[["voted_up"]]

X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=57, test_size = 0.5,stratify=y)

In [21]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
ScoreModel(clf, X_train, y_train)

Accuracy:   0.90156155637995
F1 Score:   0.8981704596162428
Recall:     0.8581173260572987
Precision:  0.9421456656056918
ROC_AUC:    0.9020811879107866


In [22]:
ScoreModel(clf, X_test, y_test)

Accuracy:   0.7979641131815045
F1 Score:   0.7863918278000729
Recall:     0.7351637107776262
Precision:  0.8452941176470589
ROC_AUC:    0.7987096486849025


In [23]:
#y = df["user_suggestion"]

#X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=57)

baseline = LogisticRegression()
baseline.fit(X_train, y_train)
ScoreModel(baseline, X_train, y_train)

Accuracy:   0.912173237856958
F1 Score:   0.9129765771926825
Recall:     0.9106412005457026
Precision:  0.9153239629756599
ROC_AUC:    0.9121915623821581


In [24]:
#vector = TfidfVectorizer(analyzer=clean_reviews, ngram_range=(2, 2))
#X = vector.fit_transform(df_test["review"])

#X_df_test = pd.DataFrame(X.toarray())
#X_df_test.columns = vector.get_feature_names()
#X_df_test.head(3)

ScoreModel(baseline, X_test, y_test)

Accuracy:   0.814095928226363
F1 Score:   0.8169229462237703
Recall:     0.819918144611187
Precision:  0.813949551379719
ROC_AUC:    0.814026809735761


In [25]:
param_grid = {'alpha': [.05,0.5, 1, 1.5, 2],
        }

In [26]:
clf = MultinomialNB()
clf.fit(X_train, y_train)


clf_gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy')
clf_gs.fit(X_train, y_train)


GridSearchCV(estimator=MultinomialNB(),
             param_grid={'alpha': [0.05, 0.5, 1, 1.5, 2]}, scoring='accuracy')

In [27]:

best_model = clf_gs.best_estimator_

In [28]:

clf_gs.best_params_

{'alpha': 0.5}

In [29]:
ScoreModel(clf_gs, X_train, y_train)

Accuracy:   0.9197653351738417
F1 Score:   0.9184067380242147
Recall:     0.8925648021828103
Precision:  0.9457896638959161
ROC_AUC:    0.9200906776760045


In [30]:
ScoreModel(clf_gs, X_test, y_test)

Accuracy:   0.8019323671497585
F1 Score:   0.794522999821013
Recall:     0.7569918144611187
Precision:  0.835969868173258
ROC_AUC:    0.8024658792975985


In [31]:
#boost_grid = XGBClassifier(random_state=57, objective="reg:logistic", use_label_encoder=False)

#param_grid = {'n_estimators': [50, 75, 100],
               # 'max_depth': [1, 3, 5, 7, 10],
               #'min_samples_leaf': [1, 3, 5, 7]}

#gridsearch = GridSearchCV(boost_grid, param_grid, scoring="accuracy", n_jobs= -1)
#gridsearch.fit(X_train, y_train)

IndentationError: unexpected indent (<ipython-input-31-325bd1c8fd4e>, line 4)

In [ ]:
#best_model2 = clf_gs.best_estimator_

In [ ]:
#clf_gs.best_params_

In [ ]:
#ScoreModel(gridsearch, X_train, y_train)

In [ ]:
#ScoreModel(gridsearch, X_test, y_test)